In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("~").expanduser() / "ldap.env"
load_dotenv(dotenv_path=env_path)

True

In [2]:
from ldap3 import (
    ALL,
    ALL_ATTRIBUTES,
    ALL_OPERATIONAL_ATTRIBUTES,
    AUTO_BIND_NO_TLS,
    NTLM,
    SUBTREE,
    Connection,
    Server,
)
from ldap3.core.exceptions import LDAPCursorError


class Ldaper:
    def __init__(self, server_name, base_dn, domain, user_name, password):
        self.server_name = server_name
        self.domain = domain
        self.user_name = user_name
        self.base_dn = base_dn
        self.password = password
        self.connect()

    def connect(self):
        self.server = Server(self.server_name, get_info=ALL)
        self.connection = Connection(
            self.server,
            user="{}\\{}".format(self.domain, self.user_name),
            password=self.password,
            authentication=NTLM,
            auto_bind=True,
        )

    def lookupDepartmentUsers(self, department_name, paged_size=500):
        user_filter = "(&(department={})(!(useraccountcontrol:=2)))".format(
            department_name
        )
        return self._rawLookup(user_filter, paged_size)

    def lookupUser(self, search_username, paged_size=5):
        user_filter = "(&(sAMAccountName={})(!(useraccountcontrol:=2)))".format(
            search_username
        )
        return self._rawLookup(user_filter, paged_size)

    def _rawLookup(self, query_filter, paged_size=1):
        searchParam = {
            "search_base": self.base_dn,
            "search_filter": query_filter,
            "search_scope": SUBTREE,
            "attributes": ALL_ATTRIBUTES,
            "paged_size": paged_size,
        }

        try:
            while True:
                result = self.connection.search(**searchParam)
                results = []
                for entry in self.connection.entries:
                    user = entry.entry_attributes_as_dict
                    results.append(user)
                return results

                cookie = conn.result["controls"]["1.2.840.113556.1.4.319"]["value"][
                    "cookie"
                ]
                if cookie:
                    searchParam["paged_cookie"] = cookie
                else:
                    break
        except ldap3.LDAPSocketSendError:
            self.connect()
            return self.lookupUser(search_username)

In [3]:
ld = Ldaper(
    server_name=os.environ["ldap_server_name"],
    base_dn=os.environ["ldap_base_dn"],
    user_name=os.environ["ldap_user_name"],
    password=os.environ["ldap_password"],
    domain=os.environ["ldap_domain"],
)

In [4]:
service_accounts = []
for user in ld.lookupDepartmentUsers("ITD"):
    if "userPrincipalName" in user and "mail" in user:
        print(user["name"], user["mail"], user["userPrincipalName"])
    else:
        service_accounts.append(user)

['Steve Walters'] ['swalters@ingham.org'] ['swalters@ingham.org']
['Desiree Cook'] ['DCook@ingham.org'] ['DCook@ingham.org']
['Jacob Willett'] ['JWillett@ingham.org'] ['JWillett@ingham.org']
['misadmin'] ['MAdministrator@ingham.org'] ['MAdministrator@ingham.org']
['Aaron'] ['jonesaa@oakgov.com'] ['aaron@mi.ingham.org']
['Nicholas Thomas'] ['ndthomas@ingham.org'] ['ndthomas@ingham.org']
['Alexys Rosales'] ['ARosales@ingham.org'] ['arosales@ingham.org']
['Michael Nolen'] ['MNolen@ingham.org'] ['MNolen@ingham.org']
['Chris Hosler'] ['CHosler@ingham.org'] ['CHosler@ingham.org']
['Long Le'] ['LLe@ingham.org'] ['LLe@ingham.org']
['Rod Villarreal'] ['rvillarreal@ingham.org'] ['RVillarreal@ingham.org']
['Lori Miles'] ['Lmiles@ingham.org'] ['Lmiles@ingham.org']
['Helpdesk'] ['H@ingham.org'] ['H@ingham.org']
['Heather Booher'] ['HBooher@ingham.org'] ['HBooher@ingham.org']
['Silly Name'] ['SW@ingham.org'] ['dp_walters@ingham.org']
['Galen Flourry'] ['GFlourry@ingham.org'] ['GFlourry@ingham.org']


In [5]:
for item in service_accounts:
    print(item["name"], item["userAccountControl"])

['misguest'] [546]
['ARCAdmin'] [66048]
['SCCM Admin'] [66048]
['SCCM Service'] [66048]
['AD Service Account, Firewall'] [66048]
['Nessus Scanner'] [66048]
['Cisco ACS AD Authentication'] [66048]
['NetVault Backup Service'] [66048]
['Backup, Veeam'] [66048]
['SUPPORT_388945a0'] [66050]
['SophosVMA'] [514]
['OnCall Tech'] [66048]
['MIS System Monitor'] [512]
['Law RAM Services'] [66048]
['FTP User'] [66048]
['LDAP USER'] [66048]
['Sophos Backup'] [66048]
['BOSS Administrator'] [66048]
['Jake'] [512]
['Wireless, IT'] [66048]
['copier user, copier user'] [66048]
['Wireless, Phones'] [66048]
['XIBOSQL'] [66048]
['Sun Backup'] [66048]
['Administrator, Server'] [66050]
['DISPLAY-IT-TEST'] [66048]
['Office Status'] [66048]
['Samuel Test'] [512]
['CiscoFirewallReadonlyAccount, Service'] [66048]
['Lori MilesTech'] [66048]
['Nack Test'] [66048]
['Animal Control Database'] [66048]
['Test Account'] [66048]
['SCCM Domain Join'] [66048]
['MSOL_46f05302be44'] [66048]
['dynamometrics_exporter Service 

In [6]:
ld.lookupUser("")[0]

IndexError: list index out of range